First, all the libraries needed in order to complete the homework are imported.

In [10]:
import pandas as pd                         #import libraries
import numpy as np
import nltk
import csv
import io
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
import json
from IPython.display import HTML, display
from tabulate import tabulate
from math import log
from numpy import linalg as LA
from heapq import *
from queries import *
from voc_ii_creation import *
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from math import radians, cos, sin, asin, sqrt
from preprocessing import *
from createmap import *
import newquery as qns
from IPython.display import HTML, display
from tabulate import tabulate

[nltk_data] Downloading package stopwords to C:\Users\Ilaria
[nltk_data]     T\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Ilaria
[nltk_data]     T\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Ilaria
[nltk_data]     T\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Ilaria
[nltk_data]     T\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Step 1: Data

The data is downloaded from the link provided and it is read by the programme.

In [6]:
df = pd.read_csv('Airbnb_Texas_Rentals.csv')#convert csv file to data frame

## Step 2: Create documents

First, the csv.file is converted to a tsv.file.

In [41]:
#convert .csv file to .tsv file
csv.writer(open('Airbnb_Texas_Rentals.tsv', 'w+', encoding = 'utf8'), delimiter='\t').writerows(csv.reader(open("Airbnb_Texas_Rentals.csv",'r',encoding="utf8")))

Then, each entry (with all the columns except for "Unnamed: 0") of the csv.file is stored in a different tsv.file.

In [ ]:
df=pd.read_csv('Airbnb_Texas_Rentals.tsv', sep = '\t') #read as dataframe with tab space
df = df.drop(['Unnamed: 0'], axis = 1) #drop "Unnamed: 0" column from dataframe
df.to_csv('Houses.tsv', mode = 'a', header= True, index = False, sep = '\t' )#create tsv.file from the the dataframe
tsvfile = open('Houses.tsv', 'r').readlines()
for i in range(len(tsvfile)):
    #create a document for each entry of the data
    open('C:/Users/Jose Manuel Camacho/Desktop/ADMHW3/documents/doc_' + str(i)  + '.tsv', 'w+',encoding="utf8").writelines(tsvfile[i])

For practical reasons regarding step 3, a tsv.file is produced for each entry of the csv.file just considering the columns title and description. 

In [109]:
df=pd.read_csv('Airbnb_Texas_Rentals.tsv', sep = '\t') #read as dataframe with tab space
df2=df[["title","description"]] #new dataframe just with columns 
df2.to_csv('Houses1.tsv', mode = 'a', header= True, index = False, sep = '\t' )#create tsv.file from the dataframe

In [4]:
tsvfile = open('Houses1.tsv', 'r').readlines() #read tsv.file 

In [111]:
for i in range(len(tsvfile)):
    #create a document for each entry of the data
    open('C:/Users/Jose Manuel Camacho/Desktop/ADMHW3/definitive/doc_' + str(i)  + '.tsv', 'w+',encoding="utf8").writelines(tsvfile[i])

## Step 3: Search engine

In this step two different search engines are implemented. These two search engines, given as input a query, return the houses that match the query. As a first common step, the documents are prepocessed removing stopwords and  punctuaction, stemming and lowercasing all the words. The documents with all the columns, as the documents with just the columns title and description are preprocessed, even though will be the latter the ones used to build the search engines.

In [ ]:
prepro_all(tsvfile) #Preprocessing documents with all the columns

In [ ]:
prepro_col(tsvfile) #Preprocessing columns with columns description and title

## 3.1) Conjunctive query

At this moment, we narrow out interest on the description and title of each document. It means that the first search
engine will evaluate queries with respect to the aforementioned information.

### 3.1.1) Create your index!

First, it is created a file named "vocabulary" that maps each word to and integer which is denoted as "term_id". In order to do so, it is defined a function called vocabulary_creation . The arguments of this function are a list of words and a dictionary that already contains a "vocabulary", which can be empty.

In [6]:
vocabulary={}
aux=0
for i in range(1,len(tsvfile)):
    #loop to generate a vocabulary from all the documents with title, description and city
    aux=((open('C:/Users/Jose Manuel Camacho/Desktop/ADMHW3/definitive_1/doci_' + str(i) +'.tsv','r',encoding='utf-8')).read()).split(' ')
    vocabulary_text=vocabulary_creation(aux,vocabulary)
    vocabulary=vocabulary_text



In [7]:
with open('vocabulary.tsv', 'w') as f: #vocabulary with all words is written in file 'vocabulary.tsv'
    json.dump(vocabulary, f) 
f.close()


In [5]:
with open('vocabulary.tsv') as f: #the vocabulary is load from a tsv.file and stored in a variable
    vocabulary_load = json.load(f)

In the next piece of code, it is created an inverted index of all the documents. An inverted index is a dictionary whose keys are the term ids of the words of the vocabulary and under each key is stored a list with all the documents that contain that term. The argument of the function is the vocabulary previously created.

In [ ]:
inverted_index_creation(vocabulary_load,tsvfile) #the inverted_index is created and store in a tsv.file

In [10]:
with open('inverted_index.tsv') as f: #read file with inverted_index and store it in a variable
    inv_indx = json.load(f)

### 3.1.2)Execute the query

In this step, the vocabulary and inverted index created before are used. The search engine will just show the entries which contains **all** the words of the queries. In the function implemented (*conjuctive_query*), the query is prepocessed in the same way as the document and it is obtained the corresponding indixes of the words from the vocabulary created. Then using the inverted index it is found the documents where are found all the words. Finally, it is shown in a table the title, description, city and url of each acommodation.

Below, it is used as example the query :"*beautiful house with garden*". 

In [7]:
conjunctive_query(vocabulary_load,inv_indx,df)

beautiful house with garden


Title,Description,City,Url
"Vintage Airstream in East Austin, T","This fantastic backyard garden oasis has been completely remodeled and is available for rent by the night. Hardwood floors, AC unit, heater, full size bed with organic cotton mattress and 600 thread count Egyptian Cotton sheets. Fully equipped bathroom with shower, small closet and pedestal sink. Small refrigerator and microwave for your convenience. Free Wi-Fi! \r\n\r\nEnjoy a private entrance onto our beautiful 1/2 acre lot. Take in the fresh air while sitting in the Adirondack Chairs right outside your door or spend the day reveling in all that Austin has to offer. We are 2 1/2 miles from downtown Austin and the U.T. campus. There is a bus stop just a short walk up the street or rent one of our bikes while you are here. Check-out time is 12:00 pm (noon) and check-in time is after 2:00 pm. \r\n\r\nNo minimum stay. Rates START at $50 per night or $300 per week. Also available for rent by the month. Please look at calendar carefully as rates are subject to increase during festivals and other high traffic times (SXSW, ACL, F1, etc.) Price increases are reflected on the calendar!\r\n\r\nWant to see more pictures??? Check us out on Instagram at #mmairstream\n\nGuests are welcome to enjoy everything our backyard has to offer. Play with the cats, give treats to the pig, throw the ball to the dogs, sing to the chickens, lounge in the hammock or adirondack chairs.\n\nWe LOVE to chat with our guests, but also understand when people are looking for a private retreat. We live in the main house and spend a lot of time on our back porch and in the yard. Come talk to us and visit whenever you want. Or ask our girls to sing for you, do a gymnastics routine, or put on a theatrical play. They love the attention.",Austin,https://www.airbnb.com/rooms/949922?location=Brazos%20River%2C%20TX
Slade Street Carriage Pool House,"This historic carriage house has been beautifully converted into a rustic cozy conveniently located space just two blocks from the town square. The cottage sits on a 100 acre property next to the century old farm house, large pool, gardens, pasture, barn, and wooded nature trails with clear running streams...all just steps away. Downstairs trundle bed sleeps two with fully equipped kitchenette, private water closet, and outdoor private shower at back door. Large front porch with dining table and rocking chair faces pool and pasture. High ceiling design accommodates extra loft space and sleeping for two. A true treasure!",Coldspring,https://www.airbnb.com/rooms/15882909?location=Cleveland%2C%20TX
Stay at an architect's gorgeous home and garden!,"A beautiful, modern home, surrounded by a beautiful garden and full of natural light!\n\nDesigned by an architect, this house is wonderfully furnished.\n\nJust miles from downtown and minutes away from the Arboretum and Domain shopping centers, this is the perfect spot for a weekend getaway or a business trip.",Austin,https://www.airbnb.com/rooms/18998883?location=Austin%2C%20TX
Unique Location! Alamo Heights - Designer Inspired,"Stylish, fully remodeled home in upscale NW – Alamo Heights Area. \n\nAmazing location - House conveniently located in quiet street, with beautiful seasoned trees, prestigious neighborhood and very close to the airport, 281, 410 loop and down-town area. \n\nFeaturing an open floor plan, original hardwood floors, 3 bedrooms, 3 FULL bathrooms + an independent garden-TV room which can sleep 2 more\n\nEuropean inspired kitchen and “top of the line” decor. Driveway can park 4 cars.",San Antonio,https://www.airbnb.com/rooms/17481455?location=Converse%2C%20TX
Ensuite room in beautiful house near Uptown/DART,"Private ground floor ensuite room in 4 storey townhouse. The room is attached to a spacious semi private living/dining room with direct access to a secluded back garden and the front door. You have your own fridge/microwave/kettle plus use of the main kitchen and living area.\n Only a 15 minute walk

It can be seen that all the results displayed have in the description or title the words "*beautiful*", "*house*" and "*garden*", since "with" has been considered as a stopword.

## 3.2) Conjuctive query & Ranking score

### 3.2.1) Inverted index

In this section, it is necessary to create a new inverted index. In this case, the inverted index is a dictionary in which each key represents the index of a word. Each key stores the list of documents in which the word is contained  and its relative tfIdf coefficient. The tfIdf coefficient for a term $t$ in a document $d$ is given by the formula:
\begin{equation}
tfIdf_{t,d}=tf_{t,d}\times Idf_{t}
\end{equation}
where $tf_{t,d}$ represents the number of ocurrences of term $t$ in document $d$ and $Idf_t$ is the inverse document frequency of term $t$. The inverse document frequency of term $t$ is equivalent to $log{\frac{N}{df_t}}$, where $N$ is the number of documents considered and $df_t$ represents the number of documents in the collection that contain a term $t$. The tfIdf values are invariant with respect to the query, for this reason they are precalculated and stored in the inverted index. 

This search engine, given a query provides the best results order regarding cosine similarity score. Specifically, it gives until the best 10 results which has a non zero cosine similirarity. The cosine similarity between the document $d_i$ and the  query $q$ is computed using the following formula:
\begin{equation}
sim(\textbf{q},\textbf{d}_{i})=\frac{\textbf{q}\cdot \textbf{d}_i }{|\textbf{d}_i||\textbf{q}|}
\end{equation}

where **q** and $\textbf{d}_{i}$ are the vectors formed by the tfIdf of each term of the query and the document *i* respectively.


In [6]:
#generate a list with all the preprocessed files with columns title, description and city
collection=[(open('C:/Users/Jose Manuel Camacho/Desktop/ADMHW3/definitive_1/doci_' + str(i) +'.tsv','r',encoding='utf-8')).read().split(' ') for i in range(1,len(tsvfile))] 


In [ ]:
#the inverted_index with a tfIdf coefficient is created and store in a tsv.file
inverted_index_creation_tfIdf(vocabulary_load,collection) 

In [8]:
with open('inverted_index_tfIdf.tsv') as f:#read file with inverted_index with coefficient tfIdf and store it in a variable
    inverted_index_tfIdf = json.load(f)

### 3.2.2) Execute the query

The sample "query" considered is "Beautiful house with garden".

In [17]:
conjunctive_query_cosine(vocabulary_load,inv_indx ,inverted_index_tfIdf,collection, df)

Beautiful house with garden


Title,Description,City,Url,Score
Stay at an architect's gorgeous home and garden!,"A beautiful, modern home, surrounded by a beautiful garden and full of natural light!\n\nDesigned by an architect, this house is wonderfully furnished.\n\nJust miles from downtown and minutes away from the Arboretum and Domain shopping centers, this is the perfect spot for a weekend getaway or a business trip.",Austin,https://www.airbnb.com/rooms/18998883?location=Cedar%20Park%2C%20TX,0.316026
Stay at an architect's gorgeous home and garden!,"A beautiful, modern home, surrounded by a beautiful garden and full of natural light!\n\nDesigned by an architect, this house is wonderfully furnished.\n\nJust miles from downtown and minutes away from the Arboretum and Domain shopping centers, this is the perfect spot for a weekend getaway or a business trip.",Austin,https://www.airbnb.com/rooms/18998883?location=Austin%2C%20TX,0.316026
Relaxing house and garden,"Three room house situated in the Hill Country of Texas near vineyards and rolling hills. Only 30 minutes to downtown Austin, 15 minutes to San Marcos, 20 minutes to beautiful Wimberley, and 15 minutes to the wineries of Texas.",Kyle,https://www.airbnb.com/rooms/2927741?location=Buda%2C%20TX,0.24062
Greeley Gardens Carriage House,This is a beautiful cozy garage apartment with a large porch overlooking the grounds that make up the Greeley Gardens estate. The Apartment has low ceilings (less than 7ft) and is ideal for travelers looking for a serene indoor/outdoor environment to relax during their travels.,Houston,https://www.airbnb.com/rooms/8301898?location=Bellaire%2C%20TX,0.23677
"Home Away from Home in Spring, TX",Beautiful 2100 sq. ft house! Three bedroom and TV room. Roomy master bedroom with spacious bathroom. Bathroom has both a shower and garden tub! Back yard has a pergola and gazebo as well as patio furniture!,Spring,https://www.airbnb.com/rooms/1521227?location=Cleveland%2C%20TX,0.212303
"The Woodlands, BEAUTIFUL HOME, 1 Floor, 2 BT, 3 BD","Attractions: The Woodlands, incredible views, golf courses, theme parks with tennis courts and lakes, pools, cycle paths, tranquility and quality of life for the family.\nYou will love my house because of its cozy spaces with high ceilings in an excellent location, beautiful views, and proximity to the great shopping areas.\nIt is a single story house with a large garden; ideal for couples, business travelers, and families with children.",Spring,https://www.airbnb.com/rooms/13065223?location=Conroe%2C%20TX,0.196603
The Vintage room in Fort Worth,"Our place is a beautiful cozy open concept house in a quiet neighborhood surrounded by nature with easy access to many shops/restaurants and close enough to the city. We are centrally located about 24 minutes from DFW airport, about 21 minutes from Fort Worth Stockyards, Water Gardens( a must see in Texas) and about 26 minutes from Six Flags.",Fort Worth,https://www.airbnb.com/rooms/18959678?location=Colleyville%2C%20TX,0.188071
The Vintage room in Fort Worth,"Our place is a beautiful cozy open concept house in a quiet neighborhood surrounded by nature with easy access to many shops/restaurants and close enough to the city. We are centrally located about 24 minutes from DFW airport, about 21 minutes from Fort Worth Stockyards, Water Gardens( a must see in Texas) and about 26 minutes from Six Flags.",Fort Worth,https://www.airbnb.com/rooms/18959678?location=Burleson%2C%20TX,0.188071
The Vintage room in Fort Worth,"Our place is a beautiful cozy open concept house in a quiet neighborhood surrounded by nature with easy access to many shops/restaurants and close enough to the city. We are centrally located about 24 minutes from DFW airport, about 21 minutes from Fort Worth Stockyards, Water Gardens( a must see in Texas) and about 26 minutes from Six Flags.",Fort Worth,https://www.airbnb.com/rooms/18959678?location=Bedford%2C%20TX,0.188071
Beautiful queen bedroom in NW Austin,"My house is close to Lakeline Mall, hig

All the results contain all the words of the query, therefore the algorithm implemented is an extension of the previous one where is considered the cosine similarity score. It has been sorted using a heap data structure to rank the documents and only the ten results with the highest score were shown. All the scores are between 0 and 1 as expected.

# Step 4: Define a new score!

Now we are going to create our personal score to evaluate the results from the query that the user is going to write.

In [ ]:
# Here we select the cloumns that contains the variables we are going to take in account for the query and to calculate our score
df_score=df[["title","description","city",'url','average_rate_per_night', 'bedrooms_count']]

First we give the chance to the user to describe what kind of house is looking for, and we make an example

In [8]:
input_ = input('What are you looking for: ')

What are you looking for: beautiful house


Then we clear the words he wrote, to find them easily

In [11]:
# This function deletes the stopwords, does the stemming on all the remaining words and returns the words we are going to look for in each document, both as a list and as a string
query_creation = qns.query_creation(input_)
l_query = query_creation[0] # I am saving the list I return before, because we need it in the next function 

Now we look for all the results, but then we calculate our personal score function, and we store just the first 10 top results we obtain, and visualize them

In [12]:
results_top_k = qns.query_results(l_query, df_score)
display(HTML(tabulate(results_top_k, headers= ['Title', 'Description','City','Url', 'Average rate per night', 
                                               'Bedrooms count', 'Score'], showindex=False, tablefmt="html")))

Which price do you want to pay per night: 40
How many bedrooms do you need: 2
In which city do you want to go: San Antonio


Title,Description,City,Url,Average rate per night,Bedrooms count,Score
House Nestled in Quiet Neighborhood,"I hope you’ll love my place because of the quiet ambiance and easy accessibility to San Antonio. My space is great for groups, families, couples, adventurers, and business travelers. Enjoy your time in a private house. You have access to the living room, kitchen and dining room, bonus room, two bedrooms, an office, and two and a half bathrooms. There is plenty of space for everyone! Best yet, the house is only 20 minutes from beautiful Downtown San Antonio/River Walk (it's a short uber ride!).",San Antonio,https://www.airbnb.com/rooms/18568735?location=Converse%2C%20TX,$65,2,3
"2 bedrooms with exotic pool, home away from home","Beautiful home with 2 bedrooms available, a brand-new deluxe pool, parking space, and located in a very quiet and safe neighborhood. Moreover, if you would like to go grocery shopping, to the pharmacy, get some gas, to the bank, to get a quick bite, to have fun in a nightlife setting, or if in case of an accident you need to get a car part you will be able to find all of these locations only 3-5 minutes away from the house. The house and rooms are a perfect fit for a stay and rest getaway.",San Antonio,https://www.airbnb.com/rooms/19131969?location=Converse%2C%20TX,$150,2,3
House Nestled in Quiet Neighborhood,"I hope you’ll love my place because of the quiet ambiance and easy accessibility to San Antonio. My space is great for groups, families, couples, adventurers, and business travelers. Enjoy your time in a private house. You have access to the living room, kitchen and dining room, bonus room, two bedrooms, an office, and two and a half bathrooms. There is plenty of space for everyone! Best yet, the house is only 20 minutes from beautiful Downtown San Antonio/River Walk (it's a short uber ride!).",San Antonio,https://www.airbnb.com/rooms/18568735?location=Cibolo%2C%20TX,$65,2,3
"Beautiful Stone Oak Home--Sleeps 4-6, 1 Low Price","Two separate bedrooms bathroom, one low price! One bedroom has a queen, the other has 2 twin beds. You may book both rooms, $60 or just one, $40. For the 5-6th guest, there is an open space with full size bed available (Extra $20). Sleeps maximum six persons total. Owner lives in the house. It is near many restaurants, a park, airport and shopping. It is beautiful, clean and comfortable! Great place for all travelers! Check out: 7am (Mon-Fri) firm, and 10 am (Sat and Sun).",San Antonio,https://www.airbnb.com/rooms/13641385?location=Bulverde%2C%20TX,$60,2,3
3Beds/1Bath-Bedrooms attached toPrivate Bath-DTown,"In a beautiful historic house located 2 miles north of downtown you'll find this urban paradise. With high ceilings, tons of natural sunlight, hardwood floors, attached bathroom, large porch/backyard and a spacious layout, the house provides a prime example of how life in SA should be lived.",San Antonio,https://www.airbnb.com/rooms/17386996?location=Alamo%20Heights%2C%20TX,$95,2,3
A Cozy Home with a Beautiful Nature Views,"This home is on the North Side of San Antonio, 3 minutes away from a great golfing course and a JW Marriott Resort and Spa. This is a great stopping place for a family and comes with all amenities from cable, internet to a laundry room and 2 bathrooms. This house is against a green belt so you can enjoy the back yard and nature.This area is safe and my neighbors are also great! Hope you enjoy your stay and more so enjoy San Antonio!",San Antonio,https://www.airbnb.com/rooms/19190311?location=Cibolo%2C%20TX,$110,1,2.7
Spacious home near Sea World and Lackland AFB,"Enjoy three large bedrooms and a loft. Entire house is for rent (3160 sq ft). Sleeps 8 comfortable. One king size bed, one queen bed, one double bed, and day bed (two twin beds). Full laundry. Near Air Force Base, Sea World (3 mi) and Six Flags and Riverwalk. Enjoy a beautiful living room, kitchen, pool table, and patio. Plenty of room for a big family or group. Fresh towels and linens. 

# Bonus Step: Make a nice visualization!

In this part we try to create a similar tool as Airbnb in order to search for the houses on the map. In order to do so, folium and geopy libraries are used. First, it is created a dataframe with  the columns which are needed to create the map(*latitude*,*longitude*,*average_rate_per_night*). The user input is a set of coordinates and a maximum distance from the coordinates. It is implemented a function for finding the distance using the Earth Surface curvature. It returns the distance of two points in Km through the computation of the great circle distance between two points on the earth (specified in decimal degrees). In the map implemented below, it is added a marker on the map for the location that the user has specified and a circle with the radius with the distance that the user introduced.

In [3]:
DataFrame = pd.read_csv("Airbnb_Texas_Rentals.csv")
columns = ['latitude', 'longitude','average_rate_per_night']
df1 = pd.DataFrame(DataFrame, columns=columns)
getcor(df1)

Enter Distance:
15
Enter coordinates:
32.7218690965595
-97.3930847419841
Number of AirBNBs: 800
Relative Number of NaN values for geographical Coordinates in the data subset: 0
